# 2-D Linear Convection with Devito

In [ ]:
from devito import Grid, TimeFunction
from examples.cfd import plot_field

nt = 100 # number of timestpes
dt = 0.005
c = 1

grid = Grid(shape=(81,81), extent=(2.,2.))

u = TimeFunction(name='u', grid=grid)
for i in range(19,40):
   u.data[0][i][19:40] = 2.

plot_field(u.data[0])

In [ ]:
from devito import Eq
from sympy import init_printing; init_printing(use_latex='mathjax')

eq = Eq(u.dt + c*u.dxl + c*u.dyl)
eq

In [ ]:
from sympy import solve

stencil = solve(eq, u.forward)
update = Eq(u.forward, stencil[0])
update

** Devito operator and solve **

We can now create the Devito operator that will generate the C code at runtime. When creating the operator, Devito's two optimization engines will log which performance optimizations have been performed:

- **DSE**: The Devito Symbolics Engine will attempt to reduce the number of operations required by the kernel.
- **DLE**: The Devito Loop Engine will perform various loop-level optimizations to improve runtime performance.

In [ ]:
from devito import Operator, configuration
configuration["log_level"] = "DEBUG"

op = Operator(update)

In [ ]:
op(time=nt+1, dt=dt)

In [ ]:
plot_field(u.data[0])